# 3 in a row RL problem

In this notebook I will try to address the 3 in a row game from a Reinforcement Learning perspective. I will try to take several basic approaches and see what works best.

## How are the elements represented?

The board is represented as a one-dimensional array. Positions are assigned as follows:

| 0 | 1 | 2 |

| 3 | 4 | 5 |

| 6 | 7 | 8 |

In [31]:
import numpy as np
import random

config = {'rows': 3, 'cols': 3, 'pieces': 3}

Task-specific functions that define the game 

In [177]:
def is_valid(board, move):
    return board[move] == 0

def is_complete(section):
    if np.all(section == 1) or np.all(section == 2):
        return section[0]
    else:
        return 0
    
def is_tie(board):
    # Assumes it has been verified that nobody wins
    return not np.count_nonzero(board == 0)
        
def who_wins(board):
    sq_board = board.reshape(config['rows'], config['cols'])
    # horizontal
    for row in range(config['rows']):
        for col in range(config['cols'] - config['pieces'] + 1):
            if is_complete(sq_board[row, col:col+config['pieces']]):
                return is_complete(sq_board[row, col:col+config['pieces']])
    # vertical
    for col in range(config['cols']):
        for row in range(config['rows'] - config['pieces'] + 1):
            if is_complete(sq_board[row:row+config['pieces'], col]):
                return is_complete(sq_board[row:row+config['pieces'], col])
    # positive diagonal
    # TODO: further develop for a non-standard case (non-3x3 board)
    if is_complete(sq_board[range(config['pieces']), range(config['pieces'] - 1, -1, -1)]):
        return is_complete(sq_board[range(config['pieces']), range(config['pieces'] - 1, -1, -1)])
    # negative diagonal
    # TODO: further develop for a non-standard case (non-3x3 board)
    if is_complete(sq_board[range(config['pieces']), range(config['pieces'])]):
        return is_complete(sq_board[range(config['pieces']), range(config['pieces'])])
    return 0

def has_ended(board):
    return who_wins(board) or is_tie(board)

def play(agent1, agent2, verbose = True):
    board = np.zeros(9, dtype = np.int64)
    actions = []
    while not who_wins(board):
        # Agent 1 plays
        action1 = agent1(board, 1)
        actions.append(action1)
        if not is_valid(board, action1):
            print('Agent 1: Invalid move')
            return 2, board, actions
        board[action1] = 1
        if who_wins(board):
            if verbose:
                print('Agent 1 wins')
            return 1, board, actions
        
        if is_tie(board):
            if verbose:
                print('Tie')
            return -1, board, actions
        
        # Agent 2 plays
        action2 = agent2(board, 2)
        actions.append(action2)
        if not is_valid(board, action2):
            print('Agent 2: Invalid move')
            return 1, board, actions
        board[action2] = 2
        
    if verbose:
        print('Agent 2 wins')
    return 2, board, actions

def draw_board(board):
    sq_board = board.reshape(config['rows'], config['cols']).tolist()    
    for line in sq_board:
        line = [' ' if piece == 0 else piece for piece in line]
        line = ['X' if piece == 1 else piece for piece in line]
        line = ['O' if piece == 2 else piece for piece in line]
        print('|' + '|'.join(line) + '|')

## Random agent
This agent takes random (valid) moves at each step. It will be used as the _baseline_, to compete against other agents and check their performance.

In [171]:
def random_agent(board, agent_code):
    valid_moves = [move for move in range(config['rows'] * config['cols']) if is_valid(board, move)]
    return random.choice(valid_moves)

In [172]:
result, board, actions = play(random_agent, random_agent)

draw_board(board)

print(actions)

Agent 1 wins
|X| |O|
|O|O| |
|X|X|X|
[6, 3, 0, 4, 7, 2, 8]


In [181]:
def get_performance(ngames, agent1, agent2 = random_agent):
    vict1 = 0
    vict2 = 0
    ties = 0
    results = []

    # Agent 1 plays first (roughly) half of the time
    for game in range(ngames//2):
        result, board, actions = play(agent1, agent2, verbose = False)
        results.append(result)
        vict1 += result == 1
        vict2 += result == 2
        ties += result == -1
    
    # Switching starting agent
    for game in range(ngames//2, ngames):
        result, board, actions = play(agent2, agent1, verbose = False)
        results.append(result)
        vict1 += result == 2
        vict2 += result == 1
        ties += result == -1
        
    print(vict1, vict2, ties)
    print('Agent 1 wins ' + str(100*vict1/ngames) + '% of the time')
    return vict1, vict2, ties, results

In [185]:
v1, v2, _, results = get_performance(1000, random_agent)

413 457 130
Agent 1 wins 41.3% of the time


## Implementing a 1-step lookahead agent
With the aim of improving performance, I will design an agent that looks one step ahead to see which move will yield the best result. In this first implementation, the only fact that it will take into account is if he is going to win in the next step. This should be enough to significantly improve the random agent's performance.

In [196]:
def get_reward(board, agent_code, move=None):
    if move is not None:
        next_board = np.copy(board)
        next_board[move] = agent_code
        return get_reward(next_board, agent_code)
    else:
        rew = 0
        h_ended = who_wins(board)
        rew += 1e3*(h_ended == agent_code)
        rew -= 1e3*(h_ended == (agent_code%2 + 1)) # Will be used in future implementations
        return rew

def agent_1step(board, agent_code):
    valid_moves = [move for move in range(config['rows'] * config['cols']) if is_valid(board, move)]
    mv_rew = [(move, get_reward(board, agent_code, move)) for move in valid_moves]
    return random.choice([mv_r[0] for mv_r in mv_rew if (mv_r[1] >= max(mv_rew, key = lambda x: x[1])[1])])

In [197]:
result, board, actions = play(agent_1step, random_agent)

draw_board(board)

print(actions)

Agent 1 wins
|O| |O|
|X|O| |
|X|X|X|
[3, 0, 8, 2, 6, 4, 7]


In [198]:
v1, v2, _, results = get_performance(1000, agent_1step)

673 253 74
Agent 1 wins 67.3% of the time


## N-step lookahead agent
We will implement a minimax algorithm in order to build an agent that calculates the outcomes N steps ahead. Due to the relatively short duration of these games, with a small N we should achieve good enough results.

In [201]:
def minimax(board, depth, maximizing_agent, agent_code):
    if depth == 0 or has_ended(board):
        return get_reward(board, agent_code)
    if maximizing_agent:
        val = -np.Inf
        valid_moves = [move for move in range(config['rows'] * config['cols']) if is_valid(board, move)]
        for move in valid_moves:
            new_board = np.copy(board)
            new_board[move] = agent_code
            val = max(val, minimax(new_board, depth-1, False, agent_code))
        return val
    else: # minimizing agent
        val = np.Inf
        valid_moves = [move for move in range(config['rows'] * config['cols']) if is_valid(board, move)]
        for move in valid_moves:
            new_board = np.copy(board)
            new_board[move] = agent_code%2 + 1
            val = min(val, minimax(new_board, depth-1, True, agent_code))
        return val
    
def get_reward_Nstep(board, agent_code, move, depth):
    next_board = np.copy(board)
    next_board[move] = agent_code
    return minimax(next_board, depth-1, False, agent_code)

In [205]:
def agent_3step(board, agent_code):
    depth = 3
    valid_moves = [move for move in range(config['rows'] * config['cols']) if is_valid(board, move)]
    mv_rew = [(move, get_reward_Nstep(board, agent_code, move, depth)) for move in valid_moves]
    return random.choice([mv_r[0] for mv_r in mv_rew if (mv_r[1] >= max(mv_rew, key = lambda x: x[1])[1])])

In [206]:
result, board, actions = play(agent_3step, random_agent)

draw_board(board)

print(actions)

Agent 1 wins
| |X|O|
| |X| |
| |X|O|
[1, 8, 4, 2, 7]


In [207]:
v1, v2, _, results = get_performance(100, agent_3step)

81 15 4
Agent 1 wins 81.0% of the time


In [208]:
def agent_5step(board, agent_code):
    depth = 5
    valid_moves = [move for move in range(config['rows'] * config['cols']) if is_valid(board, move)]
    mv_rew = [(move, get_reward_Nstep(board, agent_code, move, depth)) for move in valid_moves]
    return random.choice([mv_r[0] for mv_r in mv_rew if (mv_r[1] >= max(mv_rew, key = lambda x: x[1])[1])])

v1, v2, _, results = get_performance(100, agent_5step)

81 17 2
Agent 1 wins 81.0% of the time
